In [2]:
import os
import json
from dotenv import load_dotenv
from LLM_API import AuthenticatedRequestSender
from LLM_API import OpenAIService
from FileProcess import FileProcessor
from Memory import Embedding, LocalMemory
from Chat import JoiningChat

dotenv_path = os.path.join(os.getcwd(), '.env')
root_path = dotenv_path.rstrip('.env')
load_dotenv(dotenv_path)

service_type='sensetime'

if service_type in ['sensetime', None]:
    service = AuthenticatedRequestSender()
elif service_type == 'openai':
    service = OpenAIService()
else:
    raise ValueError('未知的服务类型')

In [1]:
import os
import json
from dotenv import load_dotenv
from Chat import JoiningChat

dotenv_path = os.path.join(os.getcwd(), '.env')
root_path = dotenv_path.rstrip('.env')
load_dotenv(dotenv_path)

service_type='sensetime'

message='Hello world!'

joiningchat=JoiningChat(service_type=service_type, dotenv_path=dotenv_path)
joiningchat.process_input(message)

{'data': {'id': '6213a9909ff7000', 'choices': [{'message': 'Hello! How can I assist you today?', 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 4, 'completion_tokens': 10, 'knowledge_tokens': 0, 'total_tokens': 14}, 'plugins': {}}}
{'data': {'id': '6213a9e7522d000', 'choices': [{'message': '"Greeting_and_Assistance_Conversation"', 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 34, 'completion_tokens': 15, 'knowledge_tokens': 0, 'total_tokens': 49}, 'plugins': {}}}
[系统]: 对话已命名为 '"Greeting_and_Assistance_Conversation"' 并保存。


'Hello! How can I assist you today?'